# Optional Spark Demonstration Notebook

This notebook provides an optional, self‑contained demonstration of how the project's data pipeline could be executed using Apache Spark instead of Pandas. It is **not required** to run the main analysis and does **not** replace the primary workflow implemented in the `notebooks/01–07` pipeline.

The purpose of this notebook is to demonstrate:

- Ability to work with distributed data frameworks (Apache Spark)
- Ability to structure data ingestion and transformation pipelines in a scalable environment
- Ability to integrate Spark into a modular project architecture
- Ability to reproduce a subset of the Alzheimer’s dataset preprocessing workflow using Spark DataFrames

This notebook is intentionally minimal and focused.  
It mirrors the **first stage** of the project pipeline: *data ingestion and basic preprocessing*, but implemented using Spark instead of Pandas.

Do **not** need Spark or Java to evaluate the project.  
This notebook exists solely to demonstrate technical breadth.


In [ ]:
import os

from pyspark.sql import SparkSession
from pyspark.sql.functions import col, when
from pyspark.sql.functions import floor


In [ ]:
# Java configuration (local execution only)
# Adjust this path if your Java 17 installation is located elsewhere
os.environ["JAVA_HOME"] = r"C:\Program Files\Eclipse Adoptium\jdk-17.0.17.10-hotspot"
os.environ["PATH"] = os.environ["JAVA_HOME"] + r"\bin;" + os.environ["PATH"]


In [ ]:
# Initialize Spark
spark = (
    SparkSession.builder
    .appName("Alzheimer_Spark_Optional")
    .config("spark.sql.shuffle.partitions", "4") 
    .config("spark.driver.memory", "4g")
    .getOrCreate()
)

spark


In [ ]:
# Load the dataset
file_path = "data/raw/nacc_alzheimers_dataset.csv"

df = (
    spark.read
    .option("header", True)
    .option("inferSchema", True)
    .csv(file_path)
)

df.printSchema()
df.show(5)


In [ ]:
# Transformation aligned with preprocessing pipeline
df_clean = (
    df
    .withColumn("age", col("age").cast("integer"))
    .withColumn("sex", when(col("sex") == "F", 1).when(col("sex") == "M", 0))
    .withColumn("diagnosis_flag", when(col("diagnosis") == "Alzheimer", 1).otherwise(0))
    .filter(col("age").isNotNull())
)

df_clean.show(5)


In [ ]:
# Compute Alzheimer diagnosis rate by age group
df_age_groups = (
    df_clean
    .withColumn("age_group", floor(col("age") / 5) * 5)
    .groupBy("age_group")
    .agg({"diagnosis_flag": "avg"})
    .orderBy("age_group")
)

df_age_groups.show()


In [ ]:
# Stop Spark
spark.stop()
